# Categorical 12.1

In [1]:
import pandas as pd
import numpy as np

In [2]:
fruits = ["apple", "orange", "apple", "apple"] * 2

In [3]:
N = len(fruits)

In [4]:
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                   columns= ['basket_id', 'fruit', 'count', 'weight'])

In [5]:
df

,basket_id,fruit,count,weight
0,0,apple,9,2.706351
1,1,orange,9,2.136690
2,2,apple,12,1.796429
3,3,apple,4,3.848775
4,4,apple,10,0.634272
5,5,orange,12,2.945126
6,6,apple,11,3.626264
7,7,apple,14,2.186311


In [6]:
fruit_cat = df['fruit'].astype('category')

In [8]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [9]:
c = fruit_cat.values

type(c)

pandas.core.arrays.categorical.Categorical

In [10]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [11]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [12]:
df['fruit'] = df['fruit'].astype('category')

In [18]:
df.dtypes

basket_id       int32
fruit        category
count           int32
weight        float64
dtype: object

### Trying another way to create Categorical

In [19]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [20]:
categories = ['foo', 'bar', 'baz']

codes = [0, 1, 2, 0, 0, 1]

In [21]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)

In [22]:
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [25]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)

In [26]:
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [29]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

## Categoricals processing

In [31]:
np.random.seed(12345)

In [32]:
draws = np.random.randn(1000)

In [33]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [34]:
bins = pd.qcut(draws, 4)

In [35]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [36]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

In [37]:
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [38]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [39]:
bins = pd.Series(bins, name='quatile')

In [41]:
results = (pd.Series(draws)
          .groupby(bins, observed=False)
          .agg(['count', 'min', 'max'])
          .reset_index())

In [42]:
results

,quatile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [43]:
results['quatile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quatile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [48]:
pd.Series(draws).groupby(bins, observed=False).agg(['count', 'min', 'max']).reset_index()

,quatile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


## Better performance

In [73]:
N = 10000000

In [74]:
draws = pd.Series(np.random.randn(N))

In [75]:
labels = pd.Series(['foo', 'bar', 'baz', 'qux']) * (N // 4)

In [76]:
categories = labels.astype('category')

In [77]:
labels.memory_usage(deep=True)

30000328

In [78]:
categories.memory_usage(deep=True)

30000504